<a href="https://colab.research.google.com/github/DanielDes/PracticasPLN/blob/master/Practica_3_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 3 Procesamiento de Lenguaje Natural
## De San Pedro Vázquez Luis Daniel

Para la ejecución de este cuaderno en colab se recomienda cambiar el entorno de ejecucion con TPU para que los tiempos de entrenamiento sean considerablemente menores.

Referencia al [ner_dataset](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus/home) usado en la práctica. Se debe subir al entorno de ejecución antes de ejecutar el cuaderno. O si se usa en un entorno local, el archivo debe estar presente en el mismo directorio del cuaderno.

In [ ]:
import pandas as pd 
import numpy as np
# Frameworks usados para la preparacion del dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
#Framework usados para la aquitectura de la red.
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional


## Preprocesamiento

Se uso una version grande del ner_dataset que tiene las siegientes características. Por un lado para tener una mejor distribucion entre las porciones de entrenamiento, validación y testeo. Y por otro esta versión se volvió más fácil de arreglar con respecto a sus valores NaN

In [ ]:
data = pd.read_csv('ner_dataset.csv',encoding='ISO-8859-1')
data = data.fillna(method = 'ffill')


word_index= 0
tag_index = 1

data.describe()

,Sentence #,Word,POS,Tag
count,1048575,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 22480,the,NN,O
freq,104,52573,145807,887908


Observamos los tags que se encontraron en el dataset

In [ ]:
data['Tag'].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

Agrupamos cada token y cada tag encontrado, ademas se añade un token padding que usaremos para rellenas las sentencias.

In [ ]:
tokens = list(data["Word"].unique())
tokens.append('<padding>')
tags = list(data["Tag"].unique())

Para cada sentencia agrupamos los tokens con sus respectivos tags.

In [ ]:
grouped = data.groupby("Sentence #")
agg_func = lambda s: [[w,t] for w,t in zip(s["Word"].values.tolist(),
                                           s["Tag"].values.tolist())]
groups = grouped.apply(agg_func)

corpus_sentences = [sentence for sentence in groups]

Creamos un diccionario para los tokens y para los tags, para obtener de manera más rápida y eficiente los índices de cada uno.

In [ ]:
word_to_index = {w : i for i, w in enumerate(tokens)}
tag_to_index = {t : i for i, t in enumerate(tags)}

Definimos hiperparámetros de la ejecucion.

In [ ]:
# Hiperparámetros

batch_size = 32
epochs = 3
max_len = 90
embedding = 40


validation_portion = 0.1
test_portion = 0.3

Obtenemos el vector de entrada de la red, primero obteniendo para cada sentencia el índice de cada token. Luego rellenamos cada sentencia con el índice del token padding hasta que el tamaño del vector sea igual al que se definió en los hiperparámetros (max_len). Esto se tiene que hacer para que la dimension de entrada a la red sea concistente. También se debe mencionar que las entradas serán en batches de 32.

In [ ]:
X = [[word_to_index[word[word_index]] for word in sentence] for sentence in corpus_sentences]

X = pad_sequences(maxlen=max_len,sequences=X,padding="post",value=word_to_index["<padding>"])

Aplicamos el mismo tratamiento para el vector de salida.

In [ ]:
y = [[tag_to_index[word[tag_index]] for word in sentence] for sentence in corpus_sentences]

y = pad_sequences(maxlen = max_len, sequences = y, padding = "post", value = tag_to_index["O"])

Para el vector de salida, categorizamos para que tengamos una colección de one-hot vector que representa cada tag. Esto va ayudar cuando usemos entropía cruzada en la red.

In [ ]:
num_tags = data['Tag'].nunique()
y = [to_categorical(i,num_classes = num_tags) for i in y] #Creamos un one-hot vector para y

Dividimos los datos en un conjunto de entrenamiento y de testeo. Por el momento no es necesario hacer la división del conjunto de validación, ya que la red puede hacer esa reservación si lo indicamos.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_portion)

## Arquitectura de la red y entrenamiento

La arquitectura de la red se compone primero de una capa de embedding, luego de una LSTM bidireccional, por para cada salida del LSTM se le aplica una capa densa, por ello se requiere usar del wrapper TimeDistributed.

En la capa de embedding es importante mencionar que se aplica una máscara que marca todo dato que sea de padding, esto es muy importante, ya que esa máscara es usada por capas posteriores aparte de que indica a la hora de hacer la optimización que datos NO se deben de tomar en cuenta. También se debe mencionar, que en la capa de LSTM bidireccional se aplica un dropout de 0.1, esto para ayudar con problemas de overfitting.

Usamos como optimizador "adam", que es un algoritmo de optimización basado en el de gradiente descendente estocástico donde el learning rate se va optimizando, y como función de riesgo la entropía cruzada.

La salida de la red es un tensor, donse para cada token de cada sentencia, tenemos las probabilidades de cada tag para ese token. Por ello en la última capa se usa como activación "softmax"

In [ ]:
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(tokens),
                  output_dim = embedding,
                  input_length = max_len, 
                  mask_zero = True)(input)

model = Bidirectional(LSTM(units = 50, 
                           return_sequences=True, 
                           recurrent_dropout=0.1))(model)

out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)

model = Model(input, out)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 90)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 90, 40)            1407160   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 90, 100)           36400     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 90, 17)            1717      
Total params: 1,445,277
Trainable params: 1,445,277
Non-trainable params: 0
_________________________________________________________________


Al entrenar, le indicamos que del conjunto de entrenamiento se use la decima parte para validación.

In [ ]:
history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=epochs,
                    validation_split=0.1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 30213 samples, validate on 3358 samples
Epoch 1/3
30213/30213 [==============================] - 145s 5ms/step - loss: 0.1677 - accuracy: 0.9672 - val_loss: 0.0673 - val_accuracy: 0.9822
Epoch 2/3
30213/30213 [==============================] - 142s 5ms/step - loss: 0.0423 - accuracy: 0.9884 - val_loss: 0.0359 - val_accuracy: 0.9895
Epoch 3/3
30213/30213 [==============================] - 144s 5ms/step - loss: 0.0270 - accuracy: 0.9921 - val_loss: 0.0320 - val_accuracy: 0.9904


## Resultados

Hacemos una evaluación de los resultados usando el conjunto de entrenamiento.

In [ ]:
results = model.evaluate(X_test,np.array(y_test),batch_size=batch_size)
print("Test Data\n Loss: {}\n Accuracy: {}".format(results[0],results[1]))

14388/14388 [==============================] - 12s 855us/step
Test Data
 Loss: 0.03067211702896848
 Accuracy: 0.9910122752189636


Por último mostramos algunas sentencias del conjunto de entrenamiento junto con las predicciones de cada token.

In [ ]:
for test_index in range(5):
  sent = ''.join([tokens[w] + " " if  tokens[w] != "<padding>" else '' for w in X_test[test_index]])
  print("--------\nSentence:\n{}\n".format(sent))
  predictions = model.predict(np.array([X_test[test_index]]))
  max_pred = np.argmax(predictions, axis=-1)
  for w,p,y in zip(X_test[test_index],max_pred[0],y_test[test_index]):
    word = tokens[w]
    if word == '<padding>':
      continue
    y_tag = np.argmax(y)
    print("Token: {:14}\t\tPred_tg: {}\t\tTag: {}".format(word,tags[p],tags[y_tag]))
  print("-------\n")

--------
Sentence:
In 1969 , an entire class of diplomats was sent to Vietnam . 

Token: In            		Pred_tg: O		Tag: O
Token: 1969          		Pred_tg: B-tim		Tag: B-tim
Token: ,             		Pred_tg: O		Tag: O
Token: an            		Pred_tg: O		Tag: O
Token: entire        		Pred_tg: O		Tag: O
Token: class         		Pred_tg: O		Tag: O
Token: of            		Pred_tg: O		Tag: O
Token: diplomats     		Pred_tg: O		Tag: O
Token: was           		Pred_tg: O		Tag: O
Token: sent          		Pred_tg: O		Tag: O
Token: to            		Pred_tg: O		Tag: O
Token: Vietnam       		Pred_tg: B-geo		Tag: B-geo
Token: .             		Pred_tg: O		Tag: O
-------

--------
Sentence:
The defendants appeared before London 's Old Bailey court Thursday via videolink from a high-security prison . 

Token: The           		Pred_tg: O		Tag: O
Token: defendants    		Pred_tg: O		Tag: O
Token: appeared      		Pred_tg: O		Tag: O
Token: before        		Pred_tg: O		Tag: O
Token: London        		Pred_tg: B-geo		Tag: B-g

## Conclusiones

Podemos observar que los resultados son bastante buenos. Con los parámetros que se definieron, no se requiere de muchas épocas para tener un buen desempeño.

A pesar de que las presiciones en el entrenamiento y en el testeo sean similares, nos da señal de que no hay un caso de overfitting. Sin embargo no significa que la red sea completamente infalible, el corpus usado es todavía una pequeña parte de todo lo que está en el mundo real.

## Referencias


https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/

https://www.tensorflow.org/guide/keras/masking_and_padding

https://www.tensorflow.org/guide/keras/rnn